### Optimization Data Additional Train

In [173]:
import pandas as pd

df_original = pd.read_csv('final-6.csv')
df1 = pd.read_csv('sensor_data_Exercise_expanded.csv', index_col=False)
df2 = pd.read_csv('sensor_data_Sleep.csv', index_col=False)
df3 = pd.read_csv('sensor_data_study.csv', index_col=False)
df4 = pd.read_csv('sensor_data_study2.csv', index_col=False)

In [174]:
df1 = df1.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df2 = df2.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df3 = df3.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])
df4 = df4.drop(columns=['sec','week','idx1','idx2','idx3','idx4','idx5','idx6','idx7','idx8','idx9','idx0'])

In [175]:
df_final = pd.concat([df_original, df1, df3, df4], axis=0)
df_final = df_final[df_final['peak']!=0]

In [176]:
df_sleep = df_final[df_final['category'] == 'sleep']
df_before = df_final[df_final['category'] != 'sleep']

df_sleep_head = df_sleep.head(500)

df_sleep_rest = df_sleep.iloc[500:]

df_before = pd.concat([df_sleep_head, df_before], axis=0)

In [177]:
df_before['category'].value_counts()

category
else        3836
study       2643
class       2167
exercise     870
sleep        500
Name: count, dtype: int64

In [178]:
df_add = df2.iloc[:int(0.8*len(df2))]
df_test = df2.iloc[int(0.8*len(df2)):]

In [179]:
df_after = pd.concat([df_final, df_add], axis=0)

In [180]:
df_after['category'].value_counts()

category
sleep       10023
else         3836
study        2643
class        2167
exercise      870
Name: count, dtype: int64

In [181]:
df_before = df_before[df_before['category'] != 'else']
df_after = df_after[df_after['category'] != 'else']

### Start LSTM model

In [182]:
# 1️⃣ LSTM 모델 구성 코드
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [194]:
# Load and preprocess data
#df = pd.read_csv('final.csv')
df = df_after

encoder = LabelEncoder()
Y = encoder.fit_transform(df['category'])
X = df.drop(columns=['category'])

Y_test = encoder.transform(df_test['category'])
X_test = df_test.drop(columns=['category'])

In [195]:
# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [196]:
# Reshape to (samples, timesteps=12, features)
X_seq = []
y_seq = []
seq_length = 12

previous_label = Y[0]

i = 0
while i <= len(X) - seq_length:
    current_label = Y[i + seq_length - 1]

    if current_label != previous_label:
        previous_label = Y[i + 1]
        i += 1
    else:
        X_seq.append(X[i:i+seq_length])
        y_seq.append(current_label)
        previous_label = current_label
        i += seq_length

X_seq = np.array(X_seq)
y_seq = to_categorical(y_seq)

In [197]:
# Reshape to (samples, timesteps=12, features)
X_t = []
y_t = []
seq_length = 12

previous_label = Y_test[0]

i = 0
while i <= len(X_test) - seq_length:
    current_label = Y_test[i + seq_length - 1]

    if current_label != previous_label:
        previous_label = Y_test[i + 1]
        i += 1
    else:
        X_t.append(X_test[i:i+seq_length])
        y_t.append(current_label)
        previous_label = current_label
        i += seq_length

X_t = np.array(X_t)
y_t = to_categorical(y_t, num_classes=4)

In [198]:
# Build model
model = Sequential()
model.add(LSTM(64, input_shape=(seq_length, X_seq.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_seq.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/Users/hongday/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [199]:
model.fit(X_seq, y_seq, epochs=20, batch_size=32, validation_data=(X_t, y_t))
model.save("activity_model.h5")

Epoch 1/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9382 - loss: 0.7207 - val_accuracy: 1.0000 - val_loss: 0.0068
Epoch 2/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9983 - loss: 0.0381 - val_accuracy: 1.0000 - val_loss: 0.0026
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0090 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 8.5964e-04
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 5.8404e-04
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 4.2398e-04
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 3.2662e-04
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 7.9581e-04 - val_accuracy

기존 수집된 데이터의 일부를 사용하는 경우 (학습에 사용 안한)

In [200]:
predictions = model.predict(X_t)

print(predictions)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Decode class labels back to original categories
predicted_categories = encoder.inverse_transform(predicted_classes)

print("Predicted categories for X_seq:")
predicted_categories

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[3.0178113e-05 5.0102249e-06 9.9994493e-01 1.9877431e-05]
 [2.9857660e-05 4.9084147e-06 9.9994576e-01 1.9475237e-05]
 [2.9981802e-05 4.9257824e-06 9.9994552e-01 1.9558358e-05]
 [2.7971661e-05 4.4829621e-06 9.9994946e-01 1.8173858e-05]
 [2.8251665e-05 4.5654092e-06 9.9994886e-01 1.8368708e-05]
 [3.0637075e-05 5.0919602e-06 9.9994409e-01 2.0193269e-05]
 [3.0468174e-05 5.0609624e-06 9.9994433e-01 2.0111420e-05]
 [2.9098901e-05 4.7510666e-06 9.9994731e-01 1.8804441e-05]
 [2.9275923e-05 4.7958006e-06 9.9994695e-01 1.8898063e-05]
 [2.9294524e-05 4.8120555e-06 9.9994695e-01 1.8991688e-05]
 [2.8736880e-05 4.6804507e-06 9.9994802e-01 1.8647977e-05]
 [2.8928416e-05 4.6913487e-06 9.9994779e-01 1.8655193e-05]
 [2.8799044e-05 4.6630876e-06 9.9994791e-01 1.8572015e-05]
 [2.8797098e-05 4.6174237e-06 9.9994802e-01 1.8615887e-05]
 [2.8981431e-05 4.6531400e-06 9.9994767e-01 1.8748740e-05]
 [2.9226272e-05 4.7451567e-06 9.9994707e-01 1.8905546e-05]
 [2.9363724e-05 4.

array(['sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep',
       'sleep', 'sleep', 'sleep', 'sleep', 'sleep', 'sleep'], dtype=object)

In [201]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

true_classes = np.argmax(y_t, axis=1)

# 디코딩
predicted_labels = encoder.inverse_transform(predicted_classes)
true_labels = encoder.inverse_transform(true_classes)

# 혼동 행렬
conf_mat = confusion_matrix(true_labels, predicted_labels, labels=encoder.classes_)

# Pandas DataFrame으로 보기 좋게 출력
conf_df = pd.DataFrame(conf_mat, index=encoder.classes_, columns=encoder.classes_)

print("🔢 혼동 행렬 (Confusion Matrix):")
print(conf_df)

🔢 혼동 행렬 (Confusion Matrix):
          class  exercise  sleep  study
class         0         0      0      0
exercise      0         0      0      0
sleep         0         0     76      0
study         0         0      0      0


In [202]:
num_classes = predictions.shape[1]

# 각 클래스별 평균 확률 계산
class_prob_means = {
    cls: np.mean(predictions[:, cls]) for cls in range(num_classes)
}

# 보기 좋게 출력
df = pd.DataFrame.from_dict(class_prob_means, orient='index', columns=['Mean_Probability'])
df.index.name = 'Class'
df = df.sort_index()
print(df)

       Mean_Probability
Class                  
0              0.000029
1              0.000005
2              0.999947
3              0.000019
